# Classification Experiment - Lexicality (Language vs. Non-language)
By: Shateesh Bhugwansing



### Experiment 1 - Logistic Regression

__Data__: EpochedEEG/20131216_1441_epo.fif (Epoched data from Emmanuil's batch script)

__Problem__: Classify language vs. Non-language epochs 

__Procedure__:
1. filter the epoched data for epochs 1-24 
2. label events by either language or non-language, using the dictionary that Emmanuil created (preprocessing/StimCodes